In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensornetwork as tn
import numpy as np
from tensorflow.keras.datasets import mnist
import time

tn.set_default_backend("numpy")

In [ ]:
batch_size = 128
epoch = 100
learning_rate = 1e-4

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
def preprosessing(X):
    X = X.reshape(-1, 28**2) / 255.0
    return np.stack([X, 1-X], axis=2)


x_train, x_test = preprosessing(x_train), preprosessing(x_test)
y_train, y_test = tf.one_hot(y_train, 10), tf.one_hot(y_test, 10)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_ds = train_ds.batch(batch_size)
test_ds = test_ds.batch(batch_size)

In [ ]:
for batch in train_ds.take(4):
    image, label = batch
    print(image.shape, label.shape)

In [ ]:
# MPS initialization
# The following code construct a MPS like this

#                        label_size
#                            |
    # bond_dim               |
# A1 --------- A2 ---------- L --------- A3 --------- An
# |            |                         |            |
# |            |                         |            |
#           feature      
def construct_MPS(bond_dim, feature, rank, label_size):
    label_site = rank // 2
    mps = tn.InfiniteMPS.random(
        d = [feature for _ in range(label_site)] + [label_size] + [feature for _ in range(rank - label_site)],
        D = [bond_dim for _ in range(rank+2)],
        dtype = np.float32
    )

    # left_region = np.stack(mps.tensors[:label_site], axis=0)
    # output_site = mps.tensors[label_site]
    # right_region = np.stack(mps.tensors[label_site+1:], axis=0)
    
    return mps
    # return nodes, connected_bonds


In [ ]:
class MPSModel(tf.keras.Model):
    def __init__(self, bond_dim, feature, rank, label_size):
        super(MPSModel, self).__init__()

        mps = construct_MPS(bond_dim, feature, rank, label_size)
        label_site = rank // 2
        self.left_region = tf.Variable(mps.tensors[:label_site], name="left", trainable=True)
        self.output_site = tf.Variable(mps.tensors[label_site], name="output", trainable=True)
        self.right_region = tf.Variable(mps.tensors[label_site+1:], name="right", trainable=True)

        self.rank = rank
        self.label_site = label_site

    def __call__(self, inputs):
        # input in size [rank, feature]
        for i in range(self.label_site):
            
        return 1 


In [ ]:
bond_dim = 3
feature = 2
rank = 28**2
label_size = 10

mpsL = MPSLayer(bond_dim, feature, rank, label_size)

In [ ]:
lnodes = mpsL.variables[0]

node_test = [tn.Node(tensor) for tensor in lnodes]

In [ ]:
class MPS(tf.Module):
    def __init__(self, bond_dim, feature, rank, label_size, name=None):
        super(MPS, self).__init__(name = name)

        # register trainable variables 
        left, output, right = construct_MPS(bond_dim, feature, rank, label_size)
        self.left_region = tf.Variable(left)
        self.output_site = tf.Variable(output)
        self.right_region = tf.Variable(right)

        # setting rest of args
        self.rank = rank
        self.label_size = label_size

    def __call__(self, input):
        # connect nodes to TNs
        nodes = []
        for l in range(rank//2):
            nodes.append(tn.Node(self.left_region[l].numpy()))
        nodes.append(tn.Node(self.output_site.numpy()))
        for r in range(rank - rank//2):
            nodes.append(tn.Node(self.right_region[l].numpy()))
        # nodes = [tn.Node(tensor) for tl in l] + [tn.Node(self.output_site)] + [tn.Node(tensor) for tr in r]
        for k in range(-1, rank):
            nodes[k].edges[2] ^ nodes[k+1][0]

        # connect input to MPS on a one-by-one basis
        # input in shape [batch, rank, feature]
        label_site = self.rank // 2
        scores = []
        for batch in input:
            pixs = [tn.Node(pix) for pix in batch]
            for i in range(label_site):
                nodes[i].edges[1] ^ pixs[i].edges[0]
            for j in range(label_site+1, self.rank+1):
                nodes[j].edges[1] ^ pixs[j-1].edges[0]
            scores = scores + [tn.contractors.greedy(nodes + pixs, ignore_edge_order=True)]
        return np.stack(scores, axis=0)


In [ ]:
bond_dim = 3
feature = 2
rank = 28**2
label_size = 10

mps = MPS(bond_dim, feature, rank, label_size)

In [ ]:
test_input = np.random.random_sample([128, 28**2, 2])

mps(test_input)

In [ ]:
a = np.random.random_sample([3,2,1])
b = np.random.random_sample([3,2,1])
c = np.stack([a, b])
print(c)
*d, = c
print(d[0].shape)

In [ ]:
# Testting by getting inner product of dummy pixels and MPS
# The result should be a rank 10 vector (tensor) for label determination

nodes = nodes + [tn.Node(np.ones(2), f'pix_{i}') for i in range(rank)]
start_time = time.time()

label_site = rank // 2
for i in range(label_site):
    bonds = bonds + [nodes[i].edges[1] ^ nodes[i+rank+1].edges[0]]
for j in range(label_site+1, rank+1):
    bonds = bonds + [nodes[j].edges[1] ^ nodes[j+rank].edges[0]]

rel = tn.contractors.auto(nodes, ignore_edge_order=True)
# rel = tn.contract_parallel
print(rel.tensor)
print(f"Runtime so far:         {int(time.time()-start_time)} sec\n")

# Looks like everything is OK

In [ ]:
# TODO auto Grad 

class mpsTrain(tf.Module):
    def __init__(self, )

In [ ]:
a = tn.Node(np.ones(2), f'block_{a}')
b = tn.Node(np.ones(2), f'block_{b}')
print(a.edges)
bond = a[0] ^ b[0]
c = tn.contract(bond)
print(a.edges)
print(c.tensor)

In [ ]:
x_train_batch[0,232,:]

In [ ]:
loggits = []

In [ ]:
tensors = [tf.random.normal(shape=(3, 2, 3)) for _ in range (2)] + [tf.random.normal(shape=(3, 5, 3))] + [tf.random.normal(shape=(3, 2, 3)) for _ in range(2)]

nodes = [tn.Node(tensor) for tensor in tensors]

for k in range (-1, 4):
    nodes[k].edges[2] ^ nodes[k+1].edges[0]
rel = tn.contractors.greedy(nodes, ignore_edge_order=True)

In [ ]:
a = tf.Node(tf.random.normal(shape=(3,2,3)))
b = tf.Node(tf.random.normal(shape=(3,2,3)))
a.edges[2] ^ b.edges[0]
c = tn.contract_between(a,b)
print(c.tensor)

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [11]:
tensors = [np.random.random_sample([3,2,3]) for _ in range (2)] + [np.random.random_sample([3,3,3])] + [np.random.random_sample([3,2,3]) for _ in range(2)]

nodes = [tn.Node(tensor) for tensor in tensors]

for k in range (-1, 4):
    nodes[k].edges[2] ^ nodes[k+1].edges[0]
# rel = tn.contractors.greedy(nodes, ignore_edge_order=True)

In [13]:
pixs = [tn.Node(np.ones(2), f'pix_{i}') for i in range(4)]
start_time = time.time()

label_site = 2
for i in range(label_site):
    nodes[i].edges[1] ^ pixs[i].edges[0]
for j in range(label_site+1, 5):
    nodes[j].edges[1] ^ pixs[j-1].edges[0]

rel = tn.contractors.greedy(nodes+pixs, ignore_edge_order=True)
# rel = tn.contract_parallel
print(rel.tensor.shape)
print(f"Runtime so far:         {int(time.time()-start_time)} sec\n")

(3,)
Runtime so far:         0 sec



In [14]:
rel.tensor

array([ 84.09298212,  62.4116624 , 136.58501253])

In [35]:
tensors_copy = tensors
pixs = [np.ones(2) for i in range(4)]

In [36]:
left = tf.Variable(tensors_copy[:2])
output = tf.Variable(tensors_copy[2])
right = tf.Variable(tensors_copy[3:])

In [37]:
ln = tn.Node(left.numpy())
rn = tn.Node(right.numpy())
on = tn.Node(output.numpy())

In [38]:
pixs = np.array(pixs)

In [39]:
pl = tn.Node(pixs[:2])
pr = tn.Node(pixs[2:])

print(pl.tensor.shape)
print(ln.tensor.shape)

(2, 2)
(2, 3, 2, 3)


In [40]:
ln.edges[0] ^ pl.edges[0]
ln.edges[2] ^ pl.edges[1]
rn.edges[0] ^ pr.edges[0]
rn.edges[2] ^ pr.edges[1]

ln.edges[1] ^ rn.edges[3]
ln.edges[3] ^ on.edges[0]
rn.edges[1] ^ on.edges[2]

rel_test = tn.contractors.greedy([ln,rn,on,pl,pr], ignore_edge_order=True)

In [41]:
rel_test.tensor

array([34.22093859, 28.40038055, 64.10668731])